# Elasticsearchを利用したサンプル検索

Elasticsearchを利用した検索などを試すためのノートブックです。

## 接続先の設定

**ESHOST**に接続先を設定します。サンプルでは、ローカルネットワーク環境にあるElasticsearchに接続しています。
接続先の各種設定については公式ドキュメントをご覧ください。
https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html

**INDEX**が接続先のインデックス名です。

In [1]:
from elasticsearch import Elasticsearch
import json

ESHOST = "http://localhost:9200"
INDEX = "esci-products"

esclient = Elasticsearch(ESHOST)

## スキーマの確認

対象となるインデックスのスキーマを確認します。

In [2]:
import json
response = esclient.indices.get(index=INDEX, pretty=True)
print(json.dumps(response.body, indent=2))

{
  "esci-products": {
    "aliases": {},
    "mappings": {
      "properties": {
        "Index": {
          "type": "long"
        },
        "product_brand": {
          "type": "keyword"
        },
        "product_bullet_point": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_color": {
          "type": "keyword"
        },
        "product_description": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_id": {
          "type": "keywo

## データの確認

検索してみる。

In [3]:
import pandas as pd

response = esclient.search(
    index=INDEX
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,Index,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,products_dense_vector
0,0,B06WVHDWVR,(エイトトウキョウ)eight tokyo サングラス スタイリッシュ スクエア メンズ レ...,【おすすめＰＯＩＮＴ】<br> ■紫外線カット率99パーセント以上。あなたの目と肌をお守りし...,eight tokyoは株式会社エイトの商標登録アイウェアブランドです。紫外線（ＵＶ）カット...,eight tokyo(エイトトウキョウ),ブラック/ライトブルー,jp,"[0.21766448020935059, 0.6630857586860657, -1.0..."
1,1,B06X17FJN7,What Say カラーレンズ ミラーレンズ サングラス クリアレンズ 伊達メガネ 全17色...,ソフト ＆ ハード ケース はランダムとなります。<br> また、弊社都合により 付属品 メ...,【 セット内容 】 ： オリジナルバンブーメガネケース・マイクロファイバークロスケース・商品...,What Say,ライト ブルー/ブラック,jp,"[-1.5359737873077393, 1.4935500621795654, -0.2..."
2,2,B071KNF11Z,アジアンサイズ 最適装着感 偏光レンズ 完全紫外線カット オリジナル スポーツサングラス PB15,<p>１、最適着用感：アジアンサイズよりフレーム形状を微調整、テンプルを長くまるく、かこまれ...,最適着用感：アジアンサイズよりフレーム形状を微調整、テンプルを長くまるく、かこまれた安定ホー...,SHD-PB,PB15,jp,"[-3.1255173683166504, 1.9866409301757812, -0.9..."
3,3,B0749LSC8N,Duduma 偏光 レンズ メンズスポーツサングラス 超軽量 UV400 紫外線をカット ／...,<b><br>商品紹介<br></b><br><b>1、Dudumaの特別なところのを発見<...,偏光レンズ:99%のUV400保護コーティング、有害な紫外線を100％ブロック、乱反射をカッ...,Duduma,ブラックマットフレーム/ブラックレンズ,jp,"[-2.078995943069458, 1.7433807849884033, 0.138..."
4,4,B079NYT27C,PAERDE サングラス メンズ 偏光レンズ 運転 軽量 uvカット 紫外線カット 軽量 運...,None,None,PAERDE,None,jp,"[-2.5723633766174316, 1.565696358680725, -1.73..."
5,5,B07BSZ4L2F,"偏光レンズ,サングラス,男女兼用 アルミニウム・マグネシウム合金 UV400紫外線カット偏光...",None,【唯一な正規販売店YAYGOD International】シンプルなデザイン：ストレスゼオ...,YAYGOD,2ブラックフレーム＆ブラックレンズ（昼間用）,jp,"[-1.1927859783172607, 1.8525850772857666, -0.5..."
6,6,B00JA5HQ9Y,007 / スカイフォール (字幕版),None,None,None,None,jp,"[1.9479371309280396, 1.0754470825195312, 1.506..."
7,7,B00JA5HVQ2,007 / 慰めの報酬 (字幕版),None,None,None,None,jp,"[0.7763017416000366, -0.12258395552635193, 2.2..."
8,8,B011QCTORK,007/ショーン・コネリー ブルーレイコレクション(6枚組) [Blu-ray],None,None,ウォルト・ディズニー・ジャパン株式会社,None,jp,"[0.23928499221801758, 2.345479965209961, 1.747..."
9,9,B012S0Z4CI,００９ノ１ ＴＨＥ ＥＮＤ ＯＦ ＴＨＥ ＢＥＧＩＮＮＩＮＧ,None,None,None,None,jp,"[-1.8685691356658936, 1.985978126525879, 0.824..."


## クエリを指定して検索


In [4]:
query = {
    "match": {
        "product_color": "ブラック"
    }
}

response = esclient.search(
    index=INDEX,
    query=query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,Index,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,products_dense_vector
0,33,B00DMW7HRW,シリコンパワー ポータブルHDD 1TB 2.5インチ USB3.0/2.0対応 PS4 動...,None,USB3.0対応の高速転送モデル\n持ち運びに便利な薄型＆スタイリッシュデザイン\n接続して...,シリコンパワー,ブラック,jp,"[-0.5986075401306152, 2.458662748336792, -0.41..."
1,54,B01M26KX7J,Lily 1/6 フィギュア用 12インチ 衣装 チャイナドレス 着替え セクシー レディー...,説明：<br> 1/6セクシーなチャイナドレスセット<br> このセットは、一般的にはCyの...,この服は1：6 （1/6 スケール）のドールに合わせて製造された着替え衣装でございます。\n...,Lily,ブラック,jp,"[-0.8149189949035645, 0.1008487194776535, 0.20..."
2,88,B089Q21H6Z,10.1インチタブレットホルスター 保護ケース 滑り止めソフトタブレットレザーケース 防損傷...,<p>互換性 </p> <p>タブレットケースは10.1インチタブレット用です。</p> <...,【互換性】 タブレットケースは10.1インチタブレット用です。\n【素材の構造】 耐久性のあ...,Meize,ブラック,jp,"[0.5301457047462463, 1.8223745822906494, -1.05..."
3,91,B01HKVFCTA,BlueField ブラック アウトドア キャンプ 大容量 防水バッグ 防水ダッフルバッグ ...,【注意事項】 ※検品は充分しておりますが、海外輸入品のため、細かな傷のついている場合がありま...,アウトドア、旅行、普段使いなど、様々なシーンでお使い頂けます。水しぶき、雨、潮風、砂ぼこり、...,BlueField,ブラック,jp,"[-3.5753273963928223, -1.9903755187988281, 1.6..."
4,96,B07KYY329D,[トラベリスト] スーツケース フレーム ストリーク 無料預入 大容量 76-20330 1...,None,スーツケースの種類:ハードケース(フレーム)\n本体サイズ:H:73cm/W:50.5cm/...,TRAVELIST(トラベリスト),ブラック,jp,"[2.4658026695251465, 1.6847590208053589, -0.36..."
5,97,B07SL1XVHX,リュックカバー80-100L サイズXXXL登山バッグ カバー 軽量 大きいサイズ 防水自転...,None,1、「買う理由」１個ブラックXXXL(80-100L)カバー 防水収納袋も付き 防水携帯ケー...,Miekision,ブラック,jp,"[-1.6353716850280762, 0.8532437086105347, 0.68..."
6,100,B07X58PKT9,スーツケース 100L 大型 超軽量 耐衝撃 静音ダブルキャスター TSAロック付 旅行出張,<b>ブランド紹介</b><p>GoPenguinはあなたの旅行パートナーです。お客様と同じ...,警告⚠️：GoPpenguin-JP以外の販売者は、GoPenguin製品の販売を許可されま...,GoPenguin,ブラック,jp,"[-0.21091923117637634, 2.1692638397216797, -0...."
7,126,B07VDQ5N5R,ゼエエ Zeee 3S 11.1V 50C 5200mAh Tプラグ付き リポバッテリー ラ...,「多い」と「少ない」<br> 「多い」とは、より長い遊び時間、より多い楽しみ、そしてより多い...,【詐欺にご注意ください】弊社はZeee ブランドオーナーで、ほかの業者に商品をいっさいご提供...,Zeee,ブラック,jp,"[-0.32149171829223633, 2.830641269683838, 1.31..."
8,127,B07Z66ZSKJ,ゼエエ Zeee 3S 11.1V 25C 1200mAh 電動ガン用 小型タミヤプラグ付き...,ゼエエ（Zeee）1200mAh 11.1V 25C 3S電動ガン用リポバッテリー <br>...,【詐欺にご注意ください】弊社はZeee ブランドオーナーで、ほかの業者に商品をいっさいご提供...,Zeee,ブラック,jp,"[-0.2819179892539978, 0.678372323513031, 0.799..."
9,141,B08278L3BZ,[PIASIN] ダブルフレア サージカルステンレス ボディピアス トンネル (ブラック 1...,内ねじ式のインターナルダブルフレア・トンネルピアス1点です。ネジが内側なのでホールを傷つけに...,内ねじ式のインターナルダブルフレア・トンネルピアス1点です。ネジが内側なのでホールを傷つけに...,PIASIN,ブラック,jp,"[0.6491118669509888, 1.2495417594909668, 0.579..."


## ベクトル検索（knn検索）

`text`の文字列をLLMのモデルを使って、ベクトルに変換し、knn検索を実行する。
インデックスにはすでに同じLLMのモデルを使用してドキュメントごとにベクトルが付与されているものとする。


In [5]:
import torch
import japanese_clip as ja_clip

text = "黒のリュックカバー"

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = ja_clip.load("rinna/japanese-clip-vit-b-16", cache_dir="/tmp/japanese_clip", device=device)
tokenizer = ja_clip.load_tokenizer()
encodings = ja_clip.tokenize(
    texts=text,
    device=device,
    tokenizer=tokenizer, # this is optional. if you don't pass, load tokenizer each time
)

with torch.no_grad():
    embeddings = model.get_text_features(**encodings)[0].tolist()
    

knn_query = {
    "field": "products_dense_vector",
    "query_vector": embeddings,
    "k": 10,
    "num_candidates": 100
}

response = esclient.search(
    index=INDEX,
    knn=knn_query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,Index,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,products_dense_vector
0,115920,B019GJGDO2,AKAUMA リュック ブラック,■AKAUMAについて：多くの人に愛用され続けるAKAUMA製品はユーザの年齢や用途を超えま...,【タイプ】: 3way\n【サイズ】縦43cm×横31cm*マチ12cm（14/15.6イン...,AKAUMA(アカウマ),ブラック,jp,"[-1.8878836631774902, 1.9254313707351685, 1.33..."
1,298811,B072N4ZCL7,Vaschy リュック ブラック,このポリエステル製のバックパックは、ダイヤモンドのような合成皮革、織物のラベル、軽量の外観の...,メイン素材: ポリエステル\nサイズ：長さ32cm 幅14cm 高さ43cm\n重さ：360g,Vaschy,ブラック,jp,"[-2.6634974479675293, 2.590059280395508, 1.120..."
2,243041,B0756DM55B,[レガートラルゴ] リュック 撥水加工ナイロン調 デイパック ブラック,None,タテ35cmxヨコ29cmxマチ14cm\nポケットの数:5(外側4/内側1)\n重量:450g,legato largo(レガートラルゴ),ブラック,jp,"[-1.2268253564834595, 1.047203779220581, 0.791..."
3,114986,B01E10FFR2,プロダクティー リュック サック PD1501 (ブラック),シンプルなデザインと大容量が魅力のフラップタイプリュック。 豊富なカラーリングで男女兼用、ど...,✅豊富なカラーリングと シンプル で おしゃれ なデザインが魅力の、スクエアー型(四角)フラ...,PRODUCTY WARRANTY,ブラック,jp,"[-1.4766793251037598, 2.121614933013916, 2.479..."
4,80806,B0794LX1M9,[リネーム] リュック スクエアミニリュック RRG70029 ブラック,None,タテ31cmxヨコ22cmxマチ9.5cm\nポケットの数:3(外側1/内側2)\n重量:390g,Rename(リネーム),ブラック,jp,"[-0.7344847321510315, 2.70257568359375, 0.6969..."
5,252426,B07Q7NB2JZ,VASTLAND(ヴァストランド) リュック リュックサック バックパック 大容量 45L ...,お出掛けがもっと楽しくなるおしゃれなリュックです。45L大容量でPCも収納できます。,【サイズ】約51×33×22cm【重量】約850g【容量】45L【耐荷重】約16kg【材質】...,ヴァストランド(VASTLAND),ブラック,jp,"[-1.4939841032028198, 1.1094324588775635, 2.08..."
6,12875,B07TX7MBCT,ナイキ(NIKE) リュックサック ブラック/アンスラサイト 15L ラン コミューター バ...,None,走行時の揺れを軽減するデザイン。通気性のある背面とハーネス涼しくさらりとした状態を保つ。上部...,NIKE(ナイキ),ブラック/アンスラサイト,jp,"[-1.27399742603302, 0.9160929918289185, 2.1817..."
7,80980,B07ZS43TKH,リュック レディース メンズ バックパック リュックサック かわいい-おしゃれ 大容量 軽量...,【おすすめポイント】 ①メイン収納はダブルファスナーで両側に大きく開いて荷物の出し入れが楽ち...,✅【 使い勝手抜群！ 進化系バックパック 】目指したのは最高の使い心地。バッグの中がごちゃご...,ＭＩＬＡ ＢＯＲＳＡ,【 2段収納モデル 】 ブラック,jp,"[-2.6022164821624756, 1.332568645477295, 0.955..."
8,287934,B01BMYTEIG,[グレゴリー] バックパック リュック 公式 マティア28 トゥルーブラック,None,タテ50cmxヨコ30cmxマチ15cm\nポケットの数:10(外側4/内側6)\n重量:7...,GREGORY(グレゴリー),トゥルーブラック,jp,"[-0.07990789413452148, 1.1598467826843262, 2.2..."
9,243109,B07SPB736F,"Frelaxy リュックカバー ファスナー付き 雨よけ ザックカバー 収納袋 (ブラック, S)",None,★「弊社だけのデザイン」：ファスナー付き、カバーを外さなくて、直接に外側から物を取り出すこと...,Frelaxy,ブラック,jp,"[-1.9906789064407349, 0.3223336935043335, 0.61..."
